In [1]:
import argparse
import math
import time

import torch
import torch.nn as nn
import numpy as np;
import importlib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [2]:
import pandas as pd

In [3]:
def evaluate(data, X, Y, model, evaluateL2, evaluateL1, batch_size):
    model.eval();
    total_loss = 0;
    total_loss_l1 = 0;
    n_samples = 0;
    predict = None;
    test = None;
    
    for X, Y in data.get_batches(X, Y, batch_size, False):
        output = model(X);
        if predict is None:
            predict = output;
            test = Y;
        else:
            predict = torch.cat((predict,output));
            test = torch.cat((test, Y));
        
        scale = data.scale.expand(output.size(0), data.m)
        total_loss += evaluateL2(output * scale, Y * scale).data[0]
        total_loss_l1 += evaluateL1(output * scale, Y * scale).data[0]
        n_samples += (output.size(0) * data.m);
    rse = math.sqrt(total_loss / n_samples)/data.rse
    rae = (total_loss_l1/n_samples)/data.rae
    
    predict = predict.data.cpu().numpy();
    Ytest = test.data.cpu().numpy();
    sigma_p = (predict).std(axis = 0);
    sigma_g = (Ytest).std(axis = 0);
    mean_p = predict.mean(axis = 0)
    mean_g = Ytest.mean(axis = 0)
    index = (sigma_g!=0);
    correlation = ((predict - mean_p) * (Ytest - mean_g)).mean(axis = 0)/(sigma_p * sigma_g);
    correlation = (correlation[index]).mean();
    return rse, rae, correlation;

In [4]:
def train(data, X, Y, model, criterion, optim, batch_size):
    model.train();
    total_loss = 0;
    n_samples = 0;
    for X, Y in data.get_batches(X, Y, batch_size, True):
        model.zero_grad();
        output = model(X);
        scale = data.scale.expand(output.size(0), data.m)
        loss = criterion(output * scale, Y * scale);
        loss.backward();
        grad_norm = optim.step();
        total_loss += loss.data[0];
        n_samples += (output.size(0) * data.m);
    return total_loss / n_samples

In [39]:
def normal_std(x):
    return x.std() * np.sqrt((len(x) - 1.)/(len(x)))

class Data_utility(object):
    # train and valid is the ratio of training set and validation set. test = 1 - train - valid
    def __init__(self, file_name, train, valid, cuda, horizon, window, normalize = 2):
        self.cuda = cuda;
        self.P = window;
        self.h = horizon
        fin = open(file_name);
        self.rawdat = np.loadtxt(fin,delimiter=';',skiprows=0);
        self.dat = np.zeros(self.rawdat.shape);
        self.n, self.m = self.dat.shape;
        self.normalize = 2
        self.scale = np.ones(self.m);
        self._normalized(normalize);
        self._split(int(train * self.n), int((train+valid) * self.n), self.n);
        
        self.scale = torch.from_numpy(self.scale).float();
        tmp = self.test[1] * self.scale.expand(self.test[1].size(0), self.m);
            
        if self.cuda:
            self.scale = self.scale.cuda();
        self.scale = Variable(self.scale);
        
        self.rse = normal_std(tmp);
        self.rae = torch.mean(torch.abs(tmp - torch.mean(tmp)));
    
    def _normalized(self, normalize):
        #normalized by the maximum value of entire matrix.
       
        if (normalize == 0):
            self.dat = self.rawdat
            
        if (normalize == 1):
            self.dat = self.rawdat / np.max(self.rawdat);
            
        #normlized by the maximum value of each row(sensor).
        if (normalize == 2):
            for i in range(self.m):
                self.scale[i] = np.max(np.abs(self.rawdat[:,i]));
                self.dat[:,i] = self.rawdat[:,i] / np.max(np.abs(self.rawdat[:,i]));
            
        
    def _split(self, train, valid, test):
        
        train_set = range(self.P+self.h-1, train);
        valid_set = range(train, valid);
        test_set = range(valid, self.n);
        self.train = self._batchify(train_set, self.h);
        self.valid = self._batchify(valid_set, self.h);
        self.test = self._batchify(test_set, self.h);
        
        
    def _batchify(self, idx_set, horizon):
        
        n = len(idx_set);
        X = torch.zeros((n,self.P,self.m));
        Y = torch.zeros((n,self.m));
        
        for i in range(n):
            end = idx_set[i] - self.h + 1;
            start = end - self.P;
            X[i,:,:] = torch.from_numpy(self.dat[start:end, :]);
            Y[i,:] = torch.from_numpy(self.dat[idx_set[i], :]);

        return [X, Y];

    def get_batches(self, inputs, targets, batch_size, shuffle=True):
        length = len(inputs)
        if shuffle:
            index = torch.randperm(length)
        else:
            index = torch.LongTensor(range(length))
        start_idx = 0
        while (start_idx < length):
            end_idx = min(length, start_idx + batch_size)
            excerpt = index[start_idx:end_idx]
            X = inputs[excerpt]; Y = targets[excerpt];
            if (self.cuda):
                X = X.cuda();
                Y = Y.cuda();  
            yield Variable(X), Variable(Y);
            start_idx += batch_size

In [86]:
class Model(nn.Module):
    def __init__(self, cuda, window, hidRNN, hidCNN,CNN_kernel, hidSkip,skip,highway_window,dropout,output_fun,data):
        super(Model, self).__init__()
        self.use_cuda = cuda
        self.P = window;
        self.m = data.m
        self.hidR = hidRNN;
        self.hidC = hidCNN;
        self.hidS = hidSkip;
        self.Ck = CNN_kernel;
        self.skip = skip;
        self.pt = (self.P - self.Ck)/self.skip
        self.hw = highway_window
        self.conv1 = nn.Conv2d(1, self.hidC, kernel_size = (self.Ck, self.m));
        self.GRU1 = nn.GRU(self.hidC, self.hidR);
        self.dropout = nn.Dropout(p = dropout);
        if (self.skip > 0):
            self.GRUskip = nn.GRU(self.hidC, self.hidS);
            self.linear1 = nn.Linear(self.hidR + self.skip * self.hidS, self.m);
        else:
            self.linear1 = nn.Linear(self.hidR, self.m);
        if (self.hw > 0):
            self.highway = nn.Linear(self.hw, 1);
        self.output = None;
        if (output_fun == 'sigmoid'):
            self.output = F.sigmoid;
        if (output_fun == 'tanh'):
            self.output = F.tanh;
 
    def forward(self, x):
        batch_size = x.size(0);
        
        #CNN
        c = x.view(-1, 1, self.P, self.m);
        c = F.relu(self.conv1(c));
        c = self.dropout(c);
        c = torch.squeeze(c, 3);
        
        # RNN 
        r = c.permute(2, 0, 1).contiguous();
        _, r = self.GRU1(r);
        r = self.dropout(torch.squeeze(r,0));

        
        #skip-rnn
        
        if (self.skip > 0):
            s = c[:,:, int(-self.pt * self.skip):].contiguous();
            s = s.view(batch_size, self.hidC, self.pt, self.skip);
            s = s.permute(2,0,3,1).contiguous();
            s = s.view(self.pt, batch_size * self.skip, self.hidC);
            _, s = self.GRUskip(s);
            s = s.view(batch_size, self.skip * self.hidS);
            s = self.dropout(s);
            r = torch.cat((r,s),1);
        
        res = self.linear1(r);
        
        #highway
        if (self.hw > 0):
            z = x[:, -self.hw:, :];
            z = z.permute(0,2,1).contiguous().view(-1, self.hw);
            z = self.highway(z);
            z = z.view(-1,self.m);
            res = res + z;
            
        if (self.output):
            res = self.output(res);
        return res;


In [7]:
class Optim(object):

    def _makeOptimizer(self):
        if self.method == 'sgd':
            self.optimizer = optim.SGD(self.params, lr=self.lr)
        elif self.method == 'adagrad':
            self.optimizer = optim.Adagrad(self.params, lr=self.lr)
        elif self.method == 'adadelta':
            self.optimizer = optim.Adadelta(self.params, lr=self.lr)
        elif self.method == 'adam':
            self.optimizer = optim.Adam(self.params, lr=self.lr)
        else:
            raise RuntimeError("Invalid optim method: " + self.method)

    def __init__(self, params, method, lr, max_grad_norm, lr_decay=1, start_decay_at=None):
        self.params = list(params)  # careful: params may be a generator
        self.last_ppl = None
        self.lr = lr
        self.max_grad_norm = max_grad_norm
        self.method = method
        self.lr_decay = lr_decay
        self.start_decay_at = start_decay_at
        self.start_decay = False

        self._makeOptimizer()

    def step(self):
        # Compute gradients norm.
        grad_norm = 0
        for param in self.params:
            grad_norm += math.pow(param.grad.data.norm(), 2)

        grad_norm = math.sqrt(grad_norm)
        if grad_norm > 0:
            shrinkage = self.max_grad_norm / grad_norm
        else:
            shrinkage = 1.

        for param in self.params:
            if shrinkage < 1:
                param.grad.data.mul_(shrinkage)

        self.optimizer.step()
        return grad_norm

    # decay learning rate if val perf does not improve or we hit the start_decay_at limit
    def updateLearningRate(self, ppl, epoch):
        if self.start_decay_at is not None and epoch >= self.start_decay_at:
            self.start_decay = True
        if self.last_ppl is not None and ppl > self.last_ppl:
            self.start_decay = True

        if self.start_decay:
            self.lr = self.lr * self.lr_decay
            print("Decaying learning rate to %g" % self.lr)
        #only decay for one epoch
        self.start_decay = False

        self.last_ppl = ppl

        self._makeOptimizer()

In [8]:
# parser = argparse.ArgumentParser(description='PyTorch Time series forecasting')
# parser.add_argument('--data', type=str, required=True, help='location of the data file')
data = r"F:\python数据\LD2011_2014\LD2011_2014.txt"
# parser.add_argument('--model', type=str, default='LSTNet',help='')
model = "LSTNet"
# parser.add_argument('--hidCNN', type=int, default=100, help='number of CNN hidden units')
hidCNN = 100                  
# parser.add_argument('--hidRNN', type=int, default=100, help='number of RNN hidden units')
hidRNN =100
# parser.add_argument('--window', type=int, default=24 * 7, help='window size')
window = 24 * 7
# parser.add_argument('--CNN_kernel', type=int, default=6,help='the kernel size of the CNN layers')
CNN_kernel = 6
# parser.add_argument('--highway_window', type=int, default=24,help='The window size of the highway component')
highway_window = 24
# parser.add_argument('--clip', type=float, default=10.,help='gradient clipping')
clip = 10
# parser.add_argument('--epochs', type=int, default=100,help='upper epoch limit')
epochs = 100
# parser.add_argument('--batch_size', type=int, default=128, metavar='N',help='batch size')
batch_size = 128
# parser.add_argument('--dropout', type=float, default=0.2,help='dropout applied to layers (0 = no dropout)')
dropout = 0.2
# parser.add_argument('--seed', type=int, default=54321,help='random seed')
seed = 54321
# parser.add_argument('--gpu', type=int, default=None)
gpu =None
# parser.add_argument('--log_interval', type=int, default=2000, metavar='N', help='report interval')
log_interval = 2000
# parser.add_argument('--save', type=str,  default='model/model.pt', help='path to save the final model')
save = 'model/model.pt'
# parser.add_argument('--cuda', type=str, default=True)
cuda = False
# parser.add_argument('--optim', type=str, default='adam')
optim = "adam"
# parser.add_argument('--lr', type=float, default=0.001)
lr = 0.001
# parser.add_argument('--horizon', type=int, default=12)
horizon = 12
# parser.add_argument('--skip', type=float, default=24)
skip = 24
# parser.add_argument('--hidSkip', type=int, default=5)
hidSkip = 5
# parser.add_argument('--L1Loss', type=bool, default=True)
L1Loss = True
# parser.add_argument('--normalize', type=int, default=2)
normalize = 2
# parser.add_argument('--output_fun', type=str, default='sigmoid')
output_fun = "sigmoid"

In [9]:
torch.manual_seed(seed)

In [101]:
Data = pd.read_csv(r"F:\python数据\LD2011_2014\LD2011_2014.txt",sep=";",index_col=0)

In [102]:
datas = (Data[["MT_002","MT_003","MT_004","MT_005"]].values)[40000:45000]

In [103]:
f = open(r"F:\python数据\LD2011_2014\data.txt","w")

In [104]:
for i in range(len(datas)):
    f.write(str(datas[i][0]) + ";"+ str(datas[i][1])+ ";"+ str(datas[i][2])+ ";"+ str(datas[i][3]))
    f.write('\n')
f.close()
print("sucess")

sucess


In [105]:
# 去除逗号
import re
with open(r"F:\python数据\LD2011_2014\data.txt") as txtData:
    lines = txtData.read()
    pat = re.compile(",")
    a = pat.sub("", lines)
    fw = open(r"F:\python数据\LD2011_2014\data2.txt","w")
    fw.write(a)
    fw.close()

In [106]:
file_data  = r"F:\python数据\LD2011_2014\data2.txt"

In [107]:
Data = Data_utility(file_data, 0.6, 0.2, False, horizon, window, normalize)
print(Data.rse)

tensor(2.4992e+14)


In [108]:
model = Model(cuda=cuda, window=window, hidRNN=hidRNN, hidCNN=hidCNN, hidSkip=hidSkip,
                          CNN_kernel=CNN_kernel,skip=skip, highway_window=highway_window, dropout=dropout,output_fun=output_fun,data=Data)

In [100]:
best_val = 10000000
optim = Optim(
    model.parameters(), optim, lr, clip,
)

AttributeError: 'str' object has no attribute 'Adam'